# 002 线性回归的简洁实现

In [1]:
# import package
import numpy as np
import torch
from torch.utils import data
from torch import nn


## 构造一个人造的数据集

### 定义线性模型参数

- Weight: $w=[2, -3.4]^T$
- Bias: $b=4.2$
- 噪声: $\epsilon$
- 标签: $y = Xw + b + \epsilon$

In [2]:
def create_data(w, b, size):
    # X: 均值为0, 方差为1, 大小为size*len(w)
    X = torch.normal(0, 1, (size, len(w)))
    y = torch.matmul(X, w) + b
    # 创造噪音epsilon
    epsilon = torch.normal(0, 0.01, y.shape)
    y += epsilon
    # reshape(x, y): 将张量变形，值为-1时表示自动.
    return X, y.reshape((-1, 1))


real_w, real_b = torch.tensor([2, -3.4]), 4.2
features, labels = create_data(w=real_w, b=real_b, size=1000)

features[0], labels[0]


(tensor([ 1.0600, -0.2760]), tensor([7.2651]))

### 调用API读取参数

In [3]:
def load_data(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


batch_size = 10
data_iter = load_data((features, labels), batch_size)

next(iter(data_iter))


[tensor([[ 0.3182,  0.2601],
         [-1.0888, -1.0948],
         [ 1.5087,  0.7775],
         [-1.1778,  1.3631],
         [-0.1293,  0.0852],
         [-1.5048,  0.6720],
         [ 1.1586,  1.2535],
         [-0.5643, -1.7203],
         [-0.0268, -0.0390],
         [-1.1336, -0.4565]]),
 tensor([[ 3.9654],
         [ 5.7510],
         [ 4.5785],
         [-2.7883],
         [ 3.6623],
         [-1.0922],
         [ 2.2574],
         [ 8.9123],
         [ 4.2725],
         [ 3.4643]])]

## 定义模型

### 使用框架预定义好的层并初始化模型参数

In [4]:
net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)


tensor([0.])

### 定义Loss函数MSELoss

In [5]:
loss = nn.MSELoss()


### 实例化SGD实例

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)


## 开始训练

In [7]:
epoch_times = 10
for epoch in range(epoch_times):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch: {epoch+1}, loss: {l:f}')



epoch: 1, loss: 0.000328
epoch: 2, loss: 0.000095
epoch: 3, loss: 0.000094
epoch: 4, loss: 0.000095
epoch: 5, loss: 0.000095
epoch: 6, loss: 0.000094
epoch: 7, loss: 0.000094
epoch: 8, loss: 0.000094
epoch: 9, loss: 0.000094
epoch: 10, loss: 0.000094


In [8]:
w = net[0].weight.data
b = net[0].bias.data

diff_w = real_w - w.reshape(real_w.shape)
diff_b = real_b - b

diff_w, diff_b

(tensor([ 0.0004, -0.0004]), tensor([0.0006]))